In [46]:
import numpy as np
import itertools
import math
from tqdm import tqdm_notebook
import pandas as pd
from scipy.sparse import dok_matrix
import matplotlib.pyplot as plt

def isExplored(s):
    s_coo = s.tocoo()
    return set(zip(s_coo.row, s_coo.col))

SV = ["S", "I", "R"]
SV0 = ["500", "1", "0"]
C = ["gamma", "rho", "mu", "beta"]
CVals = ["1.0/3.0", "1.24", "0.0", "gamma*rho"]
Phi = ["(S > 0) and (I > 0)", "I > 0", "R > 0"]
Delta = [("S = S - 1", "I = I + 1"), ("I = I - 1", "R = R + 1"), ("R = R - 1", "S = S + 1")]
Lambda = ["beta * S * I / (S+I+R)", "gamma * I", "mu * R"]
Measures = [("S", [0.0, 100.0, 5.0]), ("I", [0.0, 100.0, 5.0]), ("R", [0.0, 100.0, 5.0])]

namespace ={}

for statevar, val in zip(SV, SV0):
    namespace[statevar] = eval(val, namespace)

for constant, val in zip(C, CVals):
    namespace[constant] = eval(val, namespace)

maxsize = sum(map(int, SV0))*sum(map(int, SV0))
initialstate = tuple(map(float, SV0))

unexploredState = []
unexploredState.append(initialstate)

ssg = dok_matrix((maxsize, maxsize), dtype=np.float32)

while len(unexploredState) > 0:
    nextState = unexploredState.pop(0)
    if (not nextState in isExplored(ssg)):
        for statevar, val in zip(SV, nextState):
            namespace[statevar] = val
        enabled = [(eval(x, namespace) and (eval(y, namespace) > 0.0)) for x,y in zip(Phi, Lambda)]
        currentRates = [eval(x, namespace) if y else float(0) for x,y in zip(Lambda, enabled)]
        transitions = [x if y > 0 else None for x,y in zip(Delta, currentRates)]
        stateStep = []
        for transition in transitions:
            if (transition is not None):
                tspace = {}
                for statevar, val in zip(SV, nextState):
                    tspace[statevar] = val
                for formula in transition:
                    exec(formula, tspace)
                stateStep.append((tspace['S'], tspace['I'], tspace['R']))
            else:
                stateStep.append(None)
        print(zip(stateStep, currentRates))

[((499.0, 2.0, 0.0), 0.499001996007984), ((500.0, 0.0, 1.0), 0.3333333333333333), (None, 0.0)]


251001